In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
from sklearn.metrics import recall_score

import pandas as pd
from datetime import datetime
import os
import spacepy.pycdf as pycdf
import matplotlib.pyplot as plt
import time as tm

path = './voyager_dust_impact_4layer_model_v1.pt'
# test_cdf_path = './data/1980/vg1_pws_wf_1980-02-29T00_v1.0.cdf'
test_cdf_path = './data/1991/vg1_pws_wf_1991-04-30T09_v1.0.cdf'

In [2]:
# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1600, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.activation(self.fc4(x))
        return x

# Create an instance of the model
model = Net()
model.load_state_dict(torch.load(path))
model.eval()

Net(
  (fc1): Linear(in_features=1600, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
  (activation): Sigmoid()
)

In [3]:
# read in cdf information
test_cdf = pycdf.CDF(test_cdf_path)
time = test_cdf['Epoch'][:]
part = test_cdf['Part'][:]
sclk = test_cdf['SCLK'][:]
waveform = test_cdf['Waveform'][:]
time_offsets = test_cdf['timeOffsets'][:]
test_cdf.close()

In [4]:
# test run the cdf using the model
waveform_tensor = torch.Tensor(waveform)
with torch.no_grad():
    y_pred = model(waveform_tensor)
    y_pred_binary = torch.round(y_pred).squeeze().tolist()

# get the indices of 1s
indices = np.where(np.array(y_pred_binary) == 1)[0]
print(len(indices))

6


In [5]:
for index in indices:
    detail = f'index = {index}, sclk = {sclk[index]}, closest_dt= {time[index]}'
    print(detail)

    # Graph
    # Create an array with values ranging from -15 to 15
    x = waveform[index, :]

    # Create a figure with four subplots
    fig, axs = plt.subplots(nrows=4, figsize=(8, 10))

    # Plot a section of the array in each subplot
    axs[0].plot(x[:400], linewidth=1)
    axs[1].plot(x[400:800], linewidth=1)
    axs[2].plot(x[800:1200], linewidth=1)
    axs[3].plot(x[1200:1600], linewidth=1)

    # Set the y-axis limits to -15 and 15
    for ax in axs:
        ax.set_ylim([-15, 15])
    
    plt.text(5, 0.5, detail)

    # Show the plot
    # plt.show()
    plt.savefig(f'/Volumes/Garrys_T7/voyager-1-ml/temp/{index}.png')
    plt.close()


index = 54, sclk = 35931:40:072, closest_dt= 1991-04-30 09:05:35.210999
index = 78, sclk = 35931:40:096, closest_dt= 1991-04-30 09:05:36.651000
index = 568, sclk = 35931:40:586, closest_dt= 1991-04-30 09:06:06.051000
index = 660, sclk = 35931:40:678, closest_dt= 1991-04-30 09:06:11.570999
index = 730, sclk = 35931:40:748, closest_dt= 1991-04-30 09:06:15.771000
index = 731, sclk = 35931:40:749, closest_dt= 1991-04-30 09:06:15.830999
